In [2]:
from keras.layers import *
import tensorflow as tf
from keras import backend
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import datetime
from keras.models import Model

# **MobileNet_V1**


In [4]:
def depthwise_separable(my_input, filters_pointwise, kernel, stride):
    x = DepthwiseConv2D(kernel_size=kernel, strides=stride, padding='same')(my_input)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters_pointwise, 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    return x


def mobile_net_v1(classes):
    my_input = Input((224, 224, 3))
    x = Conv2D(32, 3, 2, padding='same')(my_input)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = depthwise_separable(x, 64, 3, 1)
    x = depthwise_separable(x, 128, 3, 2)
    x = depthwise_separable(x, 128, 3, 1)
    x = depthwise_separable(x, 256, 3, 2)
    x = depthwise_separable(x, 256, 3, 1)
    x = depthwise_separable(x, 512, 3, 2)

    for i in range(5):
      x = depthwise_separable(x, 512, 3, 1)

    x = depthwise_separable(x, 1024, 3, 2)
    x = depthwise_separable(x, 1024, 3, 1)

    x = GlobalAveragePooling2D()(x)
    x = Dense(classes, activation='softmax')(x)

    model = Model(inputs=my_input, outputs=x)

    return model


In [6]:
model = mobile_net_v1(1000)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 112, 112, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 32)     128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 112, 112, 32)      0         
                                                                 
 depthwise_conv2d (Depthwise  (None, 112, 112, 32)     320       
 Conv2D)                                                         
                                                                 
 batch_normalization_1 (Batc  (None, 112, 112, 32)     128   

# **MobileNet_V2**

In [ ]:
def bottle_neck_v2(my_input, t, filters, stride, out_channels):
    x = Conv2D(filters*t, 1, 1)(my_input)
    x = BatchNormalization()(x)
    x = ReLU(6)(x)

    x = DepthwiseConv2D(3, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU(6)(x)

    x = Conv2D(out_channels, 1, 1)(x)
    x = BatchNormalization()(x)

    if my_input.shape[-1] == x.shape[-1]:
      x = Add()([my_input, x])

    return x


def mobilenet_v2(classes):
    my_input = Input(shape=(224, 224, 3))
    x = Conv2D(32, 3, 2, padding='same')(my_input)
    x = BatchNormalization()(x)
    x = ReLU(6)(x)

    x = bottle_neck_v2(x, 1, x.shape[-1], 1, 16)

    x = bottle_neck_v2(x, 6, x.shape[-1], 2, 24)
    x = bottle_neck_v2(x, 6, x.shape[-1], 1, 24)

    x = bottle_neck_v2(x, 6, x.shape[-1], 2, 32)
    x = bottle_neck_v2(x, 6, x.shape[-1], 1, 32)
    x = bottle_neck_v2(x, 6, x.shape[-1], 1, 32)

    x = bottle_neck_v2(x, 6, x.shape[-1], 2, 64)
    x = bottle_neck_v2(x, 6, x.shape[-1], 1, 64)
    x = bottle_neck_v2(x, 6, x.shape[-1], 1, 64)
    x = bottle_neck_v2(x, 6, x.shape[-1], 1, 64)

    x = bottle_neck_v2(x, 6, x.shape[-1], 1, 96)
    x = bottle_neck_v2(x, 6, x.shape[-1], 1, 96)
    x = bottle_neck_v2(x, 6, x.shape[-1], 1, 96)

    x = bottle_neck_v2(x, 6, x.shape[-1], 2, 160)
    x = bottle_neck_v2(x, 6, x.shape[-1], 1, 160)
    x = bottle_neck_v2(x, 6, x.shape[-1], 1, 160)

    x = bottle_neck_v2(x, 6, x.shape[-1], 1, 320)

    x = Conv2D(1280, 1, 1, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU(6)(x)

    x = GlobalAveragePooling2D()(x)
    x = Dense(classes, activation='softmax')(x)

    model = Model(inputs=my_input, outputs=x)

    return model

In [ ]:
model = mobilenet_v2(10)
model.summary()

# **MobileNet_V3**

In [ ]:
def h_swish(x):
    return (x * tf.nn.relu6(x+3)) / 6


def se_block(x, ch, ratio=16):
    x_copy = x
    x = GlobalAveragePooling2D()(x)
    x = Dense(ch//ratio, activation='relu')(x)
    x = Dense(ch, activation='sigmoid')(x)
    x = Multiply()([x_copy, x])
    return x


def custom_activation(x, name):
    if name == "RE":
        # ReLU6
        x = backend.relu(x, max_value=6)
    else:
        # Hard swish
        x = x * backend.relu(x, max_value=6) / 6

    return x


def bottle_neck_v3(my_input, exp_size, out_channels, se, at, stride, kernel):
    x = Conv2D(exp_size, 1, 1)(my_input)
    x = BatchNormalization()(x)
    x = custom_activation(x, at)

    x = DepthwiseConv2D(kernel_size=kernel, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = custom_activation(x, at)

    if se == True:
      x = se_block(x, x.shape[-1])

    x = Conv2D(out_channels, 1, 1)(x)
    x = BatchNormalization()(x)

    if my_input.shape[-1] == x.shape[-1]:
      x = Add()([my_input, x])

    return x


def mobilenet_v3_large(classes):
    my_input = Input(shape=(224, 224, 3))
    x = Conv2D(16, 3, 2, padding='same')(my_input)
    x = BatchNormalization()(x)
    x = custom_activation(x, "HS")

    x = bottle_neck_v3(x, 16, 16, False, "RE", 1, 3)
    x = bottle_neck_v3(x, 64, 24, False, "RE", 2, 3)
    x = bottle_neck_v3(x, 72, 24, False, "RE", 1, 3)
    x = bottle_neck_v3(x, 72, 40, True, "RE", 2, 5)
    x = bottle_neck_v3(x, 120, 40, True, "RE", 1, 5)
    x = bottle_neck_v3(x, 120, 40, True, "RE", 1, 5)
    x = bottle_neck_v3(x, 240, 80, False, "HS", 2, 3)
    x = bottle_neck_v3(x, 200, 80, False, "HS", 1, 3)
    x = bottle_neck_v3(x, 184, 80, False, "HS", 1, 3)
    x = bottle_neck_v3(x, 184, 80, False, "HS", 1, 3)
    x = bottle_neck_v3(x, 480, 112, True, "HS", 1, 3)
    x = bottle_neck_v3(x, 672, 112, True, "HS", 1, 3)
    x = bottle_neck_v3(x, 672, 160, True, "HS", 2, 5)
    x = bottle_neck_v3(x, 960, 160, True, "HS", 1, 5)
    x = bottle_neck_v3(x, 960, 160, True, "HS", 1, 5)

    x = Conv2D(960, 1, 1, padding='same')(x)
    x = BatchNormalization()(x)
    x = custom_activation(x, "HS")

    x = GlobalAveragePooling2D()(x)
    x = Dense(1280)(x)
    x = BatchNormalization()(x)
    x = custom_activation(x, "HS")

    x = Dense(classes, activation='softmax')(x)

    model = Model(inputs=my_input, outputs=x)

    return model

In [ ]:
model = mobilenet_v3_large(10)
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_207 (Conv2D)            (None, 112, 112, 16  448         ['input_10[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_308 (Batch  (None, 112, 112, 16  64         ['conv2d_207[0][0]']             
 Normalization)                 )                                                           